In [2]:
import pandas as pd
import numpy as np
import os
import re


In [3]:
masterbuild = pd.DataFrame()
def clean_bytes(val):
    if isinstance(val, bytes):
        return str(val)[2:-1]  # str(b'xyz') => "b'xyz'" → "xyz"
    return val

for year in range(18, 24):  # range is exclusive at the end
    filename = f"/Users/danielmatten/Desktop/m/pcaudit_pub20{year}.sas7bdat"
    try:
        # Read SAS file
        df = pd.read_sas(filename)

        # Keep only relevant columns
        df.columns = df.columns.str.lower()

        # Drop missing values

        
        # Append to the main DataFrame
        masterbuild = pd.concat([masterbuild, df], ignore_index=True)
        print(f"{year} done")

    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.") #no 20
    except KeyError:
        print(f"Required columns not found in {filename}. Skipping.")

variables_to_keep = [
    'eds',
    'eds_code',
    'els',
    'ethnic',
    'fcs',
    'grade',
    'hms',
    'lea',
    'mastid',
    'mil',
    'schlcode',
    'sex',
    'swd',
    'swd_code'
]

masterbuild = masterbuild[variables_to_keep]
masterbuild.dropna(subset=['mastid'],inplace=True)

for col in masterbuild.columns:
    masterbuild[col] = masterbuild[col].apply(clean_bytes)
    print(col)
masterbuild.dropna(subset=['mastid'],inplace=True)
masterbuild.dropna(subset=['ethnic'],inplace=True)


masterbuild['grade_numeric'] = masterbuild['grade'].astype(str).str.extract('(\d+)').astype(float)

# Sort so latest grades come last (or first)
masterbuild_sorted = masterbuild.sort_values(by=['mastid', 'grade_numeric'], ascending=[True, False])

# Keep only the row with the highest grade per student
masterbuild_deduped = masterbuild_sorted.drop_duplicates(subset='mastid', keep='first')

# Optionally drop the helper column
masterbuild_deduped = masterbuild_deduped.drop(columns='grade_numeric')


masterbuild_deduped.to_csv("../data/masterbuild_master.csv",index=False)

18 done
19 done
File /Users/danielmatten/Desktop/m/pcaudit_pub2020.sas7bdat not found. Skipping.
21 done
22 done
23 done
mastid
lea
schlcode
birthdt
grade
ethnic
sex
aig
aig_code
swd
swd_code
eds
eds_code
els
els_year_exited
els_exit_group
els_first_year
migrant
hms
fcs
mil
in_fds
entry_date_fdf
entry_date
pef
pes
pey
rd3b_score
rd3b_level
rd03_score
rd03_level
ltg_rd_prof
ltg_rd_type
ltg_rd_score
ltg_rd_level
ltg_rd_accom
ltg_rd_ee
ltg_ma_prof
ltg_ma_subject
ltg_ma_type
ltg_ma_score
ltg_ma_level
ltg_ma_accom
ltg_ma_ee
ltg_sc_prof
ltg_sc_type
ltg_sc_score
ltg_sc_level
ltg_sc_accom
ltg_sc_ee
pc_rd_prof
pc_rd_type
pc_rd_score
pc_rd_level
pc_rd_date
pc_rd_accom
pc_rd_sp
pc_ma_prof
pc_ma_type
pc_ma_score
pc_ma_level
pc_ma_date
pc_ma_accom
pc_ma_sp
pc_sc_prof
pc_sc_type
pc_sc_score
pc_sc_level
pc_sc_date
pc_sc_accom
pc_sc_sp
pc_m1_prof
pc_m1_type
pc_m1_score
pc_m1_level
pc_m1_cycle
pc_m1_date
pc_m1_pe
pc_m1_accom
pc_m1_sp
pc_m1_ee
pc_bi_prof
pc_bi_type
pc_bi_score
pc_bi_level
pc_bi_cycle
pc

In [3]:
mast = pd.read_csv("../data/masterbuild_master.csv")

/var/folders/dk/2mw0y5w91dggrsnrpxb04jv80000gn/T/ipykernel_47644/1853874495.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  mast = pd.read_csv("../data/masterbuild_master.csv")
